# Criando features

- Esse notebook é o continuação do [1. EDA](1.%20EDA.ipynb)

# Ìndice
1. Resumo e objetivo.
2. [Features Básicas](#Features-Básicas).
3. [Criando Features Novas](#Criando-Features-Novas).
4. [Apêndice](#Apêndice)
5. [Referências](#Referências)

### Coisas a serem feitas:

- [ ] Colocar todas as referências.
    - [ ] [Falta a do Machine Learning Mastery.](#Referência)
    - [ ] [Referência do algoritmo](#back2)
- Colocar as outras featues.
    - [ ] Lagged.  
    - [ ] Meses.  
    - [ ] Médias. 
- [ ] Explicar Lagged.
    - Explicar o descarte dos Nan's.

In [57]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [58]:
path = 'data'

# Carregando os arquivos

In [59]:
train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')

# Features Básicas

#### A feature categórica shop_name, também possui informação sobre localização e tipos dos shoppings.

- Адыгея ТЦ "Мега"         => Adygea shopping center "Mega"	
- Волжский ТЦ "Волга Молл" => Volzhsky shopping center "Volga Mall"
- Вологда ТРЦ "Мармелад"   => Shopping center Vologda "Marmalade"	
- Якутск ТЦ "Центральный"  => Centro comercial Yakutsk "Central"	
- Ярославль ТЦ "Альтаир"   => Centro comercial Yaroslavl "Altair"

In [60]:
# Corrigindo uma entrada.
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

# Separando as palavras (split) e escolhendo a primeira (map(lambda x: x[0])).
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])

#Corringindo um nome.
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

print('Nome das cidades:\n',shops['city'].head())

Nome das cidades:
 0      Якутск
1      Якутск
2      Адыгея
3    Балашиха
4    Волжский
Name: city, dtype: object


> ### Obs.:
> Note que é possível retirar mais informações dos nomes, como o tipo de estabelecimento (Shopping, Centro comercial, e etc), o tipo do shopping (Volga Mall) e até mesmo a localização na cidade (Central).

<a name="back1"></a> Apesar de termos as localizações separadas, a maioria dos algoritmos só enchergam números, sendo o XGboost uma excessão. Então, temos que rotular cada nome de cidade com um número.

Deixei um exemplo simples [aqui](#go1) para quem ainda está começando.

In [61]:
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis='columns', inplace=True)
shops.head()

,shop_id,city_code
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


#### O mesmo procedimento vai ser realizado para os nomes das categorias. Vamos dividi-las em dois tipo e subtipo, como por exemplo:
    - Аксессуары - PS2 => Tipo: Acessórios / Subtipo: PS2.
    
Da mesma forma iremos colocar rótulos em cada um dos tipos e subtipos.

In [62]:
# Separando as duas categorias.
items_cat['split'] = items_cat['item_category_name'].str.split('-')

# Criando uma coluna só com o tipo e depois criando os rótulos.
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])

# Criando uma coluna só com o subtipo e depois criando os rótulos.
items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])

items_cat.head()

,item_category_name,item_category_id,split,type,type_code,subtype,subtype_code
0,PC - Гарнитуры/Наушники,0,"[PC , Гарнитуры/Наушники]",PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,"[Аксессуары , PS2]",Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,"[Аксессуары , PS3]",Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,"[Аксессуары , PS4]",Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,"[Аксессуары , PSP]",Аксессуары,1,PSP,13


In [63]:
# Retirando colunas que não tem mais utilidade.
items_cat.drop(['item_category_name', 'split', 'type', 'subtype'], axis='columns', inplace=True)
items_cat.head()

,item_category_id,type_code,subtype_code
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


Vou retirar duas colunas dos bancos de dados, item_name e date. As datas não vão ser precisas porque vou utilizar somente os meses e para isso já temos a coluna date_block_num. O nome dos itens provavelmente tem alguma coisa interessante, mais vai ser muito complicado separar algum informação útil de lá.

In [66]:
items.drop(['item_name'], axis=1, inplace=True)

In [67]:
train.drop(['date'], axis='columns', inplace=True)

# Criando Features Novas

Criar features é uma das coisas mais importantes a se fazer. Uma boa feature ajuda o modelo a fazer melhores previsões. Vou abordar a construção das seguintes features:

- Revenue.
- Lagged.  
- Meses.  
- Médias.

A feature mais básica de todas é a receita (Dinheiro total ganho por item vendido).

In [73]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

Uma lagged feature é obtida ao atrasarmos ou adiantarmos a sequência daquela feature. Como exemplo,

| Vendas_lag1   | Vendas   |
|----------|:------------- |
| 1 |  Nan |
| 2 |    1   |
| 3 | 2 |

a ideia por trás do de fazermos essa transformação, é dar ao algoritmo a oportunidade de fazer escolhas como:
- O número de vendas desse mês é o mesmo número do mês passado.
- Dependendo do tamanho do lag, no exemplo o lag foi de 1, é possível que ele aprenda sobre sazonalidade. Ele pode passar a entender que no Natal o número de vendas é maior.

<a name="back2"></a> Vou utilizar uma função criada por mim para produzir as lagged variables ([time_shift](#lagged)). Essa função foi inspirada em outra utilizada neste site [ML](), que é um ótimo site para se aprender sobre Machine Learning.

# Apêndice

<a name="go1"></a>No código abaixo, x vai guardar a relações entre os rótulos,
-  Ex.: a = 1, b = 2, e etc.

Isso é importante se quisermos fazer a transformação reversa.

[Voltar ao notebook](#back1)

In [49]:
x = LabelEncoder()

vetor_normal = ['a','b','c','d']

vetor_transformado = x.fit_transform(vetor_normal)

print('Categorias', vetor_normal)
print('Rótulos:', vetor_transformado)

voltando = x.inverse_transform(vetor_transformado)

print('Invertendo a transformação:', voltando)

Categorias ['a', 'b', 'c', 'd']
Rótulos: [0 1 2 3]
Invertendo a transformação: ['a' 'b' 'c' 'd']


----------------------------------------

### <a name="lagged"></a> Código para gerar e organizar as variáveis com lag.

[Voltar ao notebook](#back2)

In [72]:
def time_shift(df, lags, dropnan=True):
    
    '''
    Adiciona novas colunas deslocadas no tempo.
    Argumentos:
    
        df: Pandas DataFrame.
        
        lags: lista com o número de passos
               a ser deslocado.
               
        dropnan: (True) Retira as linhas que
                teham Nan's.
                
    Retorna:
        Pandas DataFrame com as colunas deslocadas.
    '''
    lags.append(0)
    lags = -1 * np.sort(lags)
    
    columns = [str(df.name)] if type(df) is pd.Series else df.columns
    col = []
    
    for column in columns:
        for lag in lags: 
            
            if lag == 0:
                new_column = column 
                col.append(new_column)
            else:
                new_column = column + '_lag(' + str(-1 * lag) + ')'
                col.append(new_column)
                df[new_column] = df[column].shift(lag)
                
    if dropnan:
        df.dropna(inplace=True)
        
    return df[col]

----------------------------------------

In [74]:
def fit_column_scaler_min_max(minval, maxval, dataframe):
    
    columns = dataframe.columns
    
    scaler = { column : MinMaxScaler(feature_range=(minval, maxval))
              for column in columns}
    
    scaler_fit = [scaler[column].fit_transform(dataframe[[column]])
                  for column in columns]
    
    for idx, column in enumerate(columns):
        dataframe[column] = scaler_fit[idx]
        
    return dataframe, scaler

# Referências
1. **Muitas das features craidas neste notebook foram retiradas deste [kernel](https://www.kaggle.com/dlarionov/feature-engineering-xgboost).**
2. **Site de onde tirei as informações sobre lagged variables. [Machine Learning Mastery]()**